# Working with Global Observables

A model with Poisson($N_\mathrm{SR} | \mu \times$S + B ) $\times$ Poisson($N_\mathrm{CR} | \tau\times$B)  with $N_\mathrm{CR}$=200, $S$=10, tau=10 constant

This will represent a signal region SR and a control region CR (that measures only the bkg)

The constant parameter tau represents the sideband scale factor

In this model a different technical approach is taken to describing the measured event count in the control region $N_\mathrm{CR}$. Here it is described as a *Global Observable*, which hides it as an explicit member of the dataset, but retains it as an implicit member of the data that is defined in the model, rather than in the data.

 * Generate an observed dataset $N_\mathrm{SR}=25(N_\mathrm{CR}=200$ is not part of the data, but is implicityly defined in the model)
 * Construct a unique statistical problem definition (RooStats::ModelConfig) that specifies
   * model = model
   * observable = $N_\mathrm{SR}$
   * global observable = $N_\mathrm{SR}$
   * parameter of interest = $\mu$
   * nuisance parameter = $B$
 * Store everything in a root file for later use
 
The concept of global observables is mostly useful for measurements where the observed value is always the same (unlike the $N_\mathrm{CR}$ of this example, which could genuinely be different in every measurement). It is commonly used in the description of subsidiary measurements that represent systematic uncertainties, where the observed value is by construction always exactly zero (for additive uncertainties) or one (for multiplicative uncertainties)

In [1]:
%jsroot

Construct an empy workspace that will contain the model

In [2]:
RooWorkspace w("w") ;


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



## Construct a Poisson signal region model

Construct expression for expected event count $N^{\exp}_\mathrm{SR} = \mu \times S + B$, with $S$ fixed and $B$ floating

In [3]:
w.factory("expr::Nexp_SR('mu*S+B',mu[1,-1,10],S[10],B[20,0,200])") ;

Construct Poisson probability model for $N^\mathrm{obs}_\mathrm{SR}$ given $N^\mathrm{exp}_\mathrm{SR}$

In [4]:
w.factory("Poisson::model_SR(Nobs_SR[0,100],Nexp_SR)") ;

## Construct a Poisson control region model 

Construct expression for expected event count $N^\mathrm{exp}_\mathrm{CR} = \tau \times B$, with $\tau$ fixed. The factor $\tau$ expresses the ratio of the CR size w.r.t. the SR size

In [5]:
w.factory("expr::Nexp_CR('tau*B',tau[10],B)") ;

Construct Poisson probability model for $N_\mathrm{obs}$ given $N_\mathrm{exp}$

*NB: If we treat Nobs_CR as a global observable, we make it constant in model.*
*This is implicitly done here by not not specifying a range, but just a single value* `x[lo,hi]` $\rightarrow $ `x[val]`

In [6]:
w.factory("Poisson::model_CR(Nobs_CR[200],Nexp_CR)") ;

## Construct a joint model for the SR and CR

In [7]:
w.factory("PROD::model(model_SR,model_CR)") ;

## Construct a dataset containing a single observed event count

Construct an empty dataset for observable Nobs

In [8]:
RooDataSet d("d","d",*w.var("Nobs_SR")) ;

Set Nobs to desired observed event count

In [9]:
w.var("Nobs_SR")->setVal(25) ;

Add observation of 25 events to dataset

In [10]:
d.add(*w.var("Nobs_SR")) ;
w.import(d,RooFit::Rename("observed_data")) ;

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset d
[#1] INFO:ObjectHandling -- RooWorkSpace::import(w) changing name of dataset from  d to observed_data


*NB: When treating Nobs_CR as a global observable, it does _not_ appear in the dataset it is implicitly part of the observed data (as *global observable*) but not _explicitly_ part of the dataset anymore

## Construct a unique statistical interpretatation of the model to be used by RooStats

Create an empty ModelConfig

In [11]:
RooStats::ModelConfig mc("ModelConfig",&w);

Define the pdf, the parameter of interest and the observables

In [12]:
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("mu"));
mc.SetNuisanceParameters(*w.var("B"));
mc.SetObservables(*w.var("Nobs_SR"));
mc.SetGlobalObservables(*w.var("Nobs_CR"));

Define the current value $\mu=1$ as an hypothesis 

In [13]:
mc.SetSnapshot(*w.var("mu"));
mc.Print();


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (B)
Global Observables:      RooArgSet:: = (Nobs_CR)
PDF:                     RooProdPdf::model[ model_SR * model_CR ] = 0.00144134
Snapshot:                
  1) 0x7f3fe895c900 RooRealVar:: mu = 1  L(-1 - 10)  "mu"



import model in the workspace

In [14]:
w.import(mc) ;
w.writeToFile("../workspaces/model.root") ;
w.pdf("model")->fitTo(d,RooFit::GlobalObservables(*w.var("Nobs_CR"))) ;

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_Nobs_SR with 1 entries
[#1] INFO:Minization --  Including the following contraint terms in minimization: (model_CR)
[#1] INFO:Minization -- The following global observables have been defined: (Nobs_CR)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_d_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (model_SR)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 B            2.00000e+01  1.00000e+01    0.00000e+00  2.00000e+02
     2 mu           1.00000e+00  1.00000e+00   -1.00000e+00  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 *********